# MoBioFP - Fingerphoto Recognition with U-Net Fingertip Extraction

## Import Python libraries

In [ ]:
import cv2
import imutils
import matplotlib.pyplot as plt

from mobiofp import utils, models
from pathlib import Path
from ipyfilechooser import FileChooser

## Define global constants

In [ ]:
DATA_DIR = "../data/raw/samples"
MODEL_CHECKPOINT_PATH = "../models/unet297-v1/arm64/weights/best.h5"

fc = FileChooser(DATA_DIR)
display(fc)

## Read RGB sample image

In [ ]:
IMAGE_PATH = str(fc.selected)

image = cv2.imread(IMAGE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = imutils.rotate_bound(image, 90)

plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title(f"Original Image: {image.shape}")
plt.axis("off")
plt.show()

## Fingertip segmentation using U-Net pre-trained model

In [ ]:
model = models.Segment()
model.load(MODEL_CHECKPOINT_PATH)
model.info()
mask = model.predict(image)

plt.figure(figsize=(15, 15))
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title(f"Original Image: {image.shape}")
plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(mask, cmap="gray")
plt.title("Predicted Fingertip Mask")
plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(image)
plt.imshow(mask, cmap="jet", alpha=0.5)
plt.title("Mask Overlay")
plt.axis("off")
plt.tight_layout()
plt.show()

## Fingertip ROI extraction

In [ ]:
bbox = utils.extract_roi(mask)
bbox_image = cv2.rectangle(image.copy(), bbox, (255, 0, 0), 5)
cv2.putText(
    bbox_image,
    "Fingertip",
    (bbox[0], bbox[1] - 10),
    cv2.FONT_HERSHEY_SIMPLEX,
    2.5,
    (255, 0, 0),
    5,
)

plt.figure(figsize=(10, 10))
plt.imshow(bbox_image)
plt.title("Fingertip Bounding Box")
plt.axis("off")
plt.show()

fingertip = utils.crop_image(image, bbox)
fingertip_mask = utils.crop_image(mask, bbox)

## Fingertip Enhancement (Grayscale conversion, Normalize, Bilateral Filter, CLAHE)

In [ ]:
# Grayscale conversion
fingertip = cv2.cvtColor(fingertip, cv2.COLOR_RGB2GRAY)

utils.plot_img_hist(fingertip, title="Fingertip Histogram")

In [ ]:
# Normalize image
fingertip = cv2.normalize(fingertip, None, 0, 255, cv2.NORM_MINMAX)

# Apply bilateral filter
fingertip = cv2.bilateralFilter(fingertip, 7, 50, 50)

# Apply CLAHE
fingertip = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(fingertip)

utils.plot_img_hist(fingertip, title="Bilateral Filter + CLAHE")

## Fingertip Binarization

In [ ]:
# Remove background
fingertip = cv2.bitwise_and(fingertip, fingertip, mask=fingertip_mask)

# Apply Mean Adaptive Threshold
binary = cv2.adaptiveThreshold(
    fingertip, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 2
)

images = [fingertip, binary]
titles = ["Fingertip (no background)", "Threshold"]

plt.figure(figsize=(5, 5))
for i, (img, title) in enumerate(zip(images, titles)):
    plt.subplot(1, 2, i + 1)
    plt.imshow(img, cmap="gray")
    plt.title(title)
    plt.axis("off")
plt.tight_layout()
plt.show()

## Convert fingerphoto (fingertip) to fingerprint and enhance features using Gabor filters

The function `to_fingerprint()` takes an fingertip-enhanced and converts it into a fingerprint-enhanced image.
It does this by:

- Resizing the image.
- Calculating the local gradient of the image using Sobel filters.
- Calculating the orientation of the ridges in the fingerprint.
- Extracting a region of the image and smoothing it to reduce noise.
- Calculating the x-signature of the region and finding its local maxima to estimate the ridge period.
- Creating a bank of Gabor filters with different orientations.
- Filtering the image with each filter in the bank.
- Assembling the final result by taking the corresponding convolution result for each pixel based on the closest orientation in the Gabor bank.
- Converting the result to grayscale.

In [ ]:
fingerprint = utils.to_fingerprint(binary)
fingerprint = utils.enhance_fingerprint(fingerprint)

## Export fingerprint-enhanced image

In [ ]:
PROCESSED_DIR = "../data/processed/unet"
fingerprint_filename = Path(IMAGE_PATH).stem + ".png"
fingerprint_filepath = PROCESSED_DIR + "/" + fingerprint_filename

print(f"Saving fingerprint to {fingerprint_filepath}")
_ = cv2.imwrite(fingerprint_filepath, fingerprint)